#### eval関数によるメタプログラミング

In [1]:
data="""
id,val,class
1,4,A
2,39,B
3,44,C
"""

"id,val,class\n1,4,A\n2,39,B\n3,44,C\n"

In [2]:
function new_struct(fields::Vector{Tuple{String,DataType}})
    name =  "A" * string(hash(fields), base=16)
    code = "begin\nstruct $name\n"
    for field in fields
        code *= field[1]*"::"*string(field[2])*"\n"
    end
    eval(Meta.parse(code * "end\n$name\nend"))
end

new_struct (generic function with 1 method)

In [3]:
MyS = new_struct([("a", Int), ("b", String), ("c", Int)])
dump(MyS)

A5cc33285fde10b33 <: Any
  a::Int64
  b::String
  c::Int64


In [4]:
function parse_data(data::AbstractString)
    lines = filter(x->length(x)>0, strip.(split(data, ('\n', '\r'))))
    colnames = string.(split(lines[1], ','))
    row1=split(lines[2], ',')
    coltypes = [occursin(r"^-?\d+$", val) ? Int64 : String for val in row1]
            
    (lines[2:end], new_struct(collect(zip(colnames, coltypes))))
end

parse_data (generic function with 1 method)

In [5]:
dump(parse_data("col1,col2,col3\nabc,123,123.5")[2])

A39a03927131d3fa0 <: Any
  col1::String
  col2::Int64
  col3::String


In [6]:
function parse_text(data::AbstractString)
    lines, MyStruct = parse_data(data)
    res = MyStruct[]
    for line in lines
        colvals = split(line, ',')
        f = (t, v)->t<:Int ? parse(Int, v) : string(v)
        vals = f.(MyStruct.types, colvals)
        push!(res, Base.invokelatest(MyStruct, vals...))
    end
    return res
end

parse_text (generic function with 1 method)

In [7]:
parse_text(data)

3-element Array{A56d02402a5387976,1}:
 A56d02402a5387976(1, 4, "A") 
 A56d02402a5387976(2, 39, "B")
 A56d02402a5387976(3, 44, "C")

#### ASTによるメタプログラミング

In [8]:
function new_struct2(fields::Vector{Tuple{String,DataType}})
    name =  "A" * string(hash(fields), base=16)
    c = Expr(:block,
            Expr(:struct,false,Symbol(name),
                Expr(:block, [Expr(:(::), Symbol(f[1]),
                                f[2]) for f in fields]...)),
            Symbol(name))
    println(c)
    eval(c)
end

new_struct2 (generic function with 1 method)

In [9]:
MyS2 = new_struct2([("a", Int), ("b", String), ("c", Int)])
dump(MyS2)

begin
    struct A5cc33285fde10b33
        a::Int64
        b::String
        c::Int64
    end
    A5cc33285fde10b33
end
A5cc33285fde10b33 <: Any
  a::Int64
  b::String
  c::Int64


In [10]:
MyS == MyS2

true

In [11]:
function new_struct3(fields::Vector{Tuple{String,DataType}})
    name =  "A" * string(hash(fields), base=16)
    c = :(begin
            struct $(Symbol(name))
               $([:($(Symbol(f[1]))::$(f[2])) for f in fields]...)
            end
            $(Symbol(name))
        end)
    println(c)
    eval(c)
end

new_struct3 (generic function with 1 method)

In [12]:
MyS3 = new_struct3([("a", Int), ("b", String), ("c", Int)])
dump(MyS3)

begin
    #= In[11]:4 =#
    struct A5cc33285fde10b33
        #= In[11]:5 =#
        a::Int64
        b::String
        c::Int64
    end
    #= In[11]:7 =#
    A5cc33285fde10b33
end
A5cc33285fde10b33 <: Any
  a::Int64
  b::String
  c::Int64


In [13]:
fields = [("a", Int), ("b", String), ("c", Int)]
#t = [((Symbol(f[1]))::(f[2])) for f in $fields]
    :(:struct, false, Symbol("tmp"),
        (:block, $[:($(Symbol(f[1]))::$(f[2])) for f in fields]))


:((:struct, false, Symbol("tmp"), (:block, Expr[:(a::Int64), :(b::String), :(c::Int64)])))

In [14]:
c1 = :($(:((:a) :: (Int64))))

:(:a::Int64)

In [15]:
fs = [[1, Int]]
c0 = :($((fs[1][1]) :: (fs[1][2])))
c0 = :($((fs[1][1]) , (fs[1][2])))
typeof(c0)

Tuple{Int64,DataType}

### 説明しよう


In [16]:
dump(VERSION)

VersionNumber
  major: UInt32 0x00000001
  minor: UInt32 0x00000002
  patch: UInt32 0x00000000
  prerelease: Tuple{} ()
  build: Tuple{} ()


In [17]:
function f1()
    eval(:(g1() = 10))
    g1() 
end

f1 (generic function with 1 method)

In [18]:
function f2()
    eval(:(g2() = 10))
    Base.invokelatest(g2)
end

f2 (generic function with 1 method)

In [19]:
f1()

MethodError: MethodError: no method matching g1()
The applicable method may be too new: running in world age 26084, while current world is 26085.
Closest candidates are:
  g1() at In[17]:2 (method too new to be called from this world context.)

In [20]:
f2()

10